# HRT (Hash Relational Tensor) - Multi-Perspective Exploration

This notebook explores HRT from multiple perspectives using the clean **IICA Architecture** (v0.4.0):

1. **HRT Basics** - Structure: AM, Lattice, Config
2. **HRT_IICA** - Immutability, Idempotence, Content Addressability
3. **LUT Embedding** - Token-to-hash mapping within HRT
4. **Merge Operations** - Algebraic properties of merge
5. **Parallel Merge** - Divide-and-conquer parallelism
6. **Persistence** - Git-like commit/push/pull
7. **HRT Stack** - In-memory history with rollback
8. **ManifoldOS_IICA** - Full orchestration workflow

---

## 1. Import Required Libraries and Modules

In [1]:
import numpy as np
import tempfile
import os
import time

# Core IICA Architecture
from core import (
    # Foundation
    HRT, HRTConfig, AdjacencyMatrix, HLLSetLattice, BasicHLLSet,
    HLLSet, Kernel,
    ImmutableTensor, compute_element_hash, compute_structural_hash,
    
    # IICA Layer
    HRT_IICA, EmbeddedLUT, LUTEntry, HRTStack, CommitInfo,
    create_hrt_iica, create_stack,
    
    # Persistence
    HRTPersistentStore, MemoryHRTStore, FileSystemHRTStore, HRTSerializer,
    CommitObject, Blob, create_memory_store, create_file_store, create_serializer,
    
    # ManifoldOS
    ManifoldOS_IICA, ManifoldIICAConfig,
    create_manifold_iica, create_parallel_manifold,
    
    # Version
    __version__
)

print(f"Fractal Manifold Core v{__version__}")
print(f"HRT_IICA: {HRT_IICA.__doc__.split(chr(10))[1].strip() if HRT_IICA.__doc__ else 'IICA-enhanced HRT'}")

Fractal Manifold Core v0.4.0
HRT_IICA: IICA-Enhanced Hash Relational Tensor.


## 2. HRT Basics - Structure Overview

HRT consists of:
- **AM (Adjacency Matrix)** - Sparse tensor of relationships
- **Lattice** - HLLSet-based probabilistic structure (rows/cols)
- **Config** - Parameters (p_bits, h_bits, tau, rho, epsilon)

In [3]:
# Create empty HRT_IICA
hrt = create_hrt_iica()

print("=== HRT_IICA Structure ===")
print(f"Name (SHA1 hash): {hrt.name[:32]}...")
print(f"Step: {hrt.step_number}")
print(f"Parent: {hrt.parent_hrt}")
print()

# Config
print("=== Config ===")
print(f"p_bits: {hrt.config.p_bits} (registers = 2^{hrt.config.p_bits} = {2**hrt.config.p_bits})")
print(f"h_bits: {hrt.config.h_bits}")
print(f"tau: {hrt.config.tau}, rho: {hrt.config.rho}")
print(f"dimension: {hrt.config.dimension}")
print()

# AM (Adjacency Matrix)
print("=== Adjacency Matrix ===")
print(f"Shape: {hrt.am.tensor.data.shape}")
print(f"Non-zero entries: {(hrt.am.tensor.data != 0).sum().item()}")
print()

# Lattice
print("=== Lattice ===")
print(f"Row basic HLLSets: {len(hrt.lattice.row_basic)}")
print(f"Col basic HLLSets: {len(hrt.lattice.col_basic)}")
print()

# LUT (embedded)
print("=== Embedded LUT ===")
print(f"Entries: {len(hrt.lut)}")
print(f"Type: {type(hrt.lut.entries).__name__} (immutable)")

=== HRT_IICA Structure ===
Name (SHA1 hash): 170cacee938b3bb506c6cd38510fc14c...
Step: 0
Parent: None

=== Config ===
p_bits: 10 (registers = 2^10 = 1024)
h_bits: 32
tau: 0.7, rho: 0.3
dimension: 32770

=== Adjacency Matrix ===
Shape: torch.Size([32770, 32770])
Non-zero entries: 0

=== Lattice ===
Row basic HLLSets: 32770
Col basic HLLSets: 32770

=== Embedded LUT ===
Entries: 0
Type: frozenset (immutable)


## 3. IICA Properties - Immutability, Idempotence, Content Addressability

**IICA** provides algebraic guarantees:
- **Immutable**: All operations return new HRT, never mutate
- **Idempotent**: `merge(A, A) == A` (same content = same hash)
- **Content Addressable**: Name is SHA1 hash of content

In [4]:
kernel = Kernel()

# Test Idempotence: merge(A, A) == A
hrt_a = create_hrt_iica()
hrt_merged = hrt_a.merge(hrt_a, kernel)

print("=== Idempotence Test ===")
print(f"HRT A name:      {hrt_a.name}")
print(f"merge(A,A) name: {hrt_merged.name}")
print(f"merge(A,A) == A: {hrt_merged.name == hrt_a.name} ✓" if hrt_merged.name == hrt_a.name else "✗")
print()

# Test Immutability: original not modified
original_name = hrt_a.name
hrt_b = create_hrt_iica()
hrt_c = hrt_a.merge(hrt_b, kernel)

print("=== Immutability Test ===")
print(f"Original HRT A unchanged: {hrt_a.name == original_name} ✓" if hrt_a.name == original_name else "✗")
print(f"New HRT C created:        {hrt_c.name != hrt_a.name}")
print()

# Content Addressability: same content = same hash
hrt_d = create_hrt_iica()  # Fresh empty HRT
hrt_e = create_hrt_iica()  # Another fresh empty HRT

print("=== Content Addressability ===")
print(f"HRT D name: {hrt_d.name}")
print(f"HRT E name: {hrt_e.name}")
print(f"Same content, same hash: {hrt_d.name == hrt_e.name} ✓" if hrt_d.name == hrt_e.name else "✗")

=== Idempotence Test ===
HRT A name:      170cacee938b3bb506c6cd38510fc14c89f7e075
merge(A,A) name: 170cacee938b3bb506c6cd38510fc14c89f7e075
merge(A,A) == A: True ✓

=== Immutability Test ===
Original HRT A unchanged: True ✓
New HRT C created:        False

=== Content Addressability ===
HRT D name: 170cacee938b3bb506c6cd38510fc14c89f7e075
HRT E name: 170cacee938b3bb506c6cd38510fc14c89f7e075
Same content, same hash: True ✓


## 4. LUT Embedding - Token Mapping

LUT (Lookup Table) is embedded in HRT_IICA. Each entry maps:
- `(register, zeros)` → token information (hash, metadata)

In [6]:
# Create HRT with LUT entries via ManifoldOS
mos = create_manifold_iica()
mos.ingest(["apple", "banana", "cherry", "date"])

print("=== Embedded LUT ===")
print(f"LUT entries: {len(mos.head.lut)}")
print()

# Inspect LUT entries (note: attribute is 'reg' not 'register')
for entry in list(mos.head.lut.entries)[:5]:
    print(f"  reg={entry.reg}, zeros={entry.zeros}")
    print(f"    token_hash={entry.token_hash}")
    print(f"    tokens={entry.tokens}")
    print()

=== Embedded LUT ===
LUT entries: 4

  reg=786, zeros=21
    token_hash=1212502802
    tokens=(('date',),)

  reg=504, zeros=16
    token_hash=61358584
    tokens=(('apple',),)

  reg=185, zeros=20
    token_hash=1001140409
    tokens=(('cherry',),)

  reg=427, zeros=22
    token_hash=2362375595
    tokens=(('banana',),)



## 5. Merge Operations - Algebraic Properties

Merge is:
- **Associative**: `merge(merge(A,B), C) == merge(A, merge(B,C))`
- **Commutative**: `merge(A,B) == merge(B,A)`
- **Idempotent**: `merge(A,A) == A`

This enables lossless parallel merge.

In [ ]:
# Create 3 distinct HRTs with different data
mos_a = create_manifold_iica()
mos_a.ingest(["alpha", "beta"])
hrt_A = mos_a.head

mos_b = create_manifold_iica()
mos_b.ingest(["gamma", "delta"])
hrt_B = mos_b.head

mos_c = create_manifold_iica()
mos_c.ingest(["epsilon", "zeta"])
hrt_C = mos_c.head

print("=== Merge Associativity Test ===")
# (A ⊕ B) ⊕ C
ab = hrt_A.merge(hrt_B, kernel)
ab_c = ab.merge(hrt_C, kernel)

# A ⊕ (B ⊕ C)
bc = hrt_B.merge(hrt_C, kernel)
a_bc = hrt_A.merge(bc, kernel)

print(f"(A ⊕ B) ⊕ C: {ab_c.name}")
print(f"A ⊕ (B ⊕ C): {a_bc.name}")
print(f"Associative: {ab_c.name == a_bc.name}")
print()

print("=== Merge Commutativity Test ===")
ab = hrt_A.merge(hrt_B, kernel)
ba = hrt_B.merge(hrt_A, kernel)
print(f"A ⊕ B: {ab.name}")
print(f"B ⊕ A: {ba.name}")
print(f"Commutative: {ab.name == ba.name}")

## 6. Parallel Merge - Divide and Conquer

Associativity enables parallel processing:
```
[A, B, C, D, E, F, G, H]  →  8 HRTs
   ↓ parallel merge pairs
[AB, CD, EF, GH]          →  4 HRTs  
   ↓ parallel merge pairs
[ABCD, EFGH]              →  2 HRTs
   ↓ final merge
[ABCDEFGH]                →  1 HRT
```

In [ ]:
# Test parallel_merge
hrts = [hrt_A, hrt_B, hrt_C]

print("=== Parallel Merge Test ===")
print(f"Input HRTs: {len(hrts)}")
for i, h in enumerate(hrts):
    print(f"  HRT[{i}]: {h.name[:24]}... LUT={len(h.lut)}")

# Use HRT_IICA.parallel_merge (divide-and-conquer)
merged = HRT_IICA.parallel_merge(hrts, kernel)

print()
print(f"Merged HRT: {merged.name[:24]}...")
print(f"Merged LUT entries: {len(merged.lut)}")
print()

# Verify: parallel merge == sequential merge
sequential = hrts[0]
for h in hrts[1:]:
    sequential = sequential.merge(h, kernel)

print(f"Sequential merge: {sequential.name[:24]}...")
print(f"Parallel == Sequential: {merged.name == sequential.name}")

## 7. Persistence - Git-like Commit/Push/Pull

HRT persistence follows Git semantics:
- **Blobs**: Content-addressed storage for AM, Lattice, LUT, Config
- **Commits**: Point to blobs, have parent chain
- **Refs**: Named pointers (branches) to commits

In [ ]:
with tempfile.TemporaryDirectory() as tmpdir:
    store_path = os.path.join(tmpdir, 'hrt_store')
    
    # Create ManifoldOS with persistent store
    mos = ManifoldOS_IICA(store_path=store_path)
    
    # Ingest data
    mos.ingest(["persistence", "test", "data", "tokens"])
    original_name = mos.head.name
    original_lut = len(mos.head.lut)
    
    print("=== Before Push ===")
    print(f"HEAD: {original_name[:32]}...")
    print(f"LUT entries: {original_lut}")
    print(f"Unpushed commits: {mos.unpushed_count}")
    
    # Push to persistent store
    mos.push()
    print()
    print("=== After Push ===")
    print(f"Unpushed commits: {mos.unpushed_count}")
    
    # Create new ManifoldOS pointing to same store (simulates restart)
    mos2 = ManifoldOS_IICA(store_path=store_path)
    
    print()
    print("=== After Reload ===")
    print(f"HEAD: {mos2.head.name[:32]}...")
    print(f"LUT entries: {len(mos2.head.lut)}")
    print(f"HEAD match: {mos2.head.name == original_name} ✓" if mos2.head.name == original_name else "✗")

## 8. HRT Stack - In-Memory History with Rollback

HRTStack provides Git-like in-memory history:
- **commit()**: Add HRT to stack
- **pop()**: Rollback to previous state
- **branch()**: Create named branch
- **switch_branch()**: Switch between branches

In [ ]:
# Test HRT Stack with rollback
mos = create_manifold_iica()

print("=== Stack Operations ===")

# Step 1
mos.ingest(["step1", "data"])
step1_name = mos.head.name
print(f"Step 1: {step1_name[:24]}... (LUT={len(mos.head.lut)})")

# Step 2
mos.ingest(["step2", "more"])
step2_name = mos.head.name
print(f"Step 2: {step2_name[:24]}... (LUT={len(mos.head.lut)})")

# Step 3
mos.ingest(["step3", "final"])
step3_name = mos.head.name
print(f"Step 3: {step3_name[:24]}... (LUT={len(mos.head.lut)})")

print()
print("=== Rollback ===")

# Rollback 1 step (from step 3 to step 2)
mos.rollback(1)
print(f"After rollback(1): {mos.head.name[:24]}...")
print(f"Back to step 2: {mos.head.name == step2_name}")

# Rollback another step (from step 2 to step 1)
mos.rollback(1)
print(f"After rollback(1): {mos.head.name[:24]}...")
print(f"Back to step 1: {mos.head.name == step1_name}")

## 9. ManifoldOS_IICA - Full Workflow with Parallel Ingestion

Complete orchestration demo with parallel batch ingestion.

In [ ]:
# Parallel ingestion with multiple workers
mos = create_parallel_manifold(workers=4)

batches = [
    ["apple", "banana", "cherry"],
    ["dog", "cat", "bird"],
    ["red", "green", "blue"],
    ["one", "two", "three"],
]

print("=== Parallel Ingestion ===")
print(f"Batches: {len(batches)}")
print(f"Workers: 4")

start = time.time()
mos.parallel_ingest(batches)
elapsed = time.time() - start

print(f"Time: {elapsed*1000:.1f}ms")
print(f"LUT entries: {len(mos.head.lut)}")
print(f"Step: {mos.step_number}")
print()

# Show some LUT entries
print("=== Sample LUT Entries ===")
for entry in list(mos.head.lut.entries)[:6]:
    print(f"  reg={entry.register:4d}, zeros={entry.zeros:2d} → {entry.source_name}")

## 10. Performance Benchmark

Compare sequential vs parallel ingestion.

In [ ]:
# Generate test batches
n_batches = 8
batch_size = 100
test_batches = [
    [f"token_{i}_{j}" for j in range(batch_size)]
    for i in range(n_batches)
]

print(f"=== Benchmark: {n_batches} batches × {batch_size} tokens ===")
print()

# Sequential
mos_seq = create_manifold_iica()
start = time.time()
for batch in test_batches:
    mos_seq.ingest(batch)
seq_time = time.time() - start
print(f"Sequential: {seq_time*1000:.1f}ms")
print(f"  LUT: {len(mos_seq.head.lut)}, Steps: {mos_seq.step_number}")

# Parallel (4 workers)
mos_par = create_parallel_manifold(workers=4)
start = time.time()
mos_par.parallel_ingest(test_batches)
par_time = time.time() - start
print(f"Parallel (4 workers): {par_time*1000:.1f}ms")
print(f"  LUT: {len(mos_par.head.lut)}, Steps: {mos_par.step_number}")

print()
if par_time < seq_time:
    print(f"Speedup: {seq_time/par_time:.2f}x faster")
else:
    print(f"Overhead from parallelism (small batches)")

## Summary

| Feature | Status |
|---------|--------|
| HRT Structure (AM, Lattice, Config) | ✓ |
| IICA Properties (Immutability, Idempotence, Content Addressable) | ✓ |
| Embedded LUT | ✓ |
| Merge (Associative, Commutative, Idempotent) | ✓ |
| Parallel Merge | ✓ |
| Git-like Persistence (push/pull) | ✓ |
| HRT Stack (rollback) | ✓ |
| ManifoldOS_IICA (full workflow) | ✓ |
| Parallel Ingestion | ✓ |

**Version**: Fractal Manifold Core v0.4.0 (IICA Architecture)